# Локальный бэктест BcsQuants v0.1
* Скачайте архив с данными https://bcsquants.com/doc/localtest и распакуйте его в директорию, в которой находится текущий файл, так чтобы файл **bcsquants.ipynb** и папка **data** были в одной директории
* Не забудьте сначала запустить ячейку с бэктестом, а потом тестировать стратегию, для этого переведите курсор на вторую ячейку и нажмите Ctrl+Enter

In [24]:
def init(self):
    self._tickSize = 'm5'
    self._window = 2
    self.alpha = 0.01 # 1 процент
    self.holdPeriod = 12 # 1 час 

def tick(self, data):
    predPrice = data['close'][0] # предпоследняя цена
    currentPrice = data['close'][1] # последняя цена

    if currentPrice >= (1 + self.alpha) * predPrice:
        order('buy', takeProfit=self.alpha, stopLoss=self.alpha/2, holdPeriod=self.holdPeriod)

    if currentPrice <= (1 - self.alpha) * predPrice:
        order('sell', takeProfit=self.alpha, stopLoss=self.alpha/2, holdPeriod=self.holdPeriod)
        
###############################################################################################
# Вставьте стратегию выше, смотри примеры https://bcsquants.com/doc/example

runBacktest(init, tick, 'SBER')

tickFile =  data/order/SBER_m5_tick.csv
orderFile = data/order/SBER_m5_order.csv


,sumProcent,maxDrawdown,std,minV,numDeals,sumTakeProfit,sumHoldPeriod,sumStopLoss
SBER_m5,-0.026124,0.021081,0.006356,-0.029192,10,0,-0.010725,-0.015399


In [22]:
# ЯЧЕЙКА С БЭКТЕСТОМ
# Это упрощенная версия бэктеста, который используется на сервере
from datetime import datetime as dt
import numpy as np
import csv
import pandas as pd

orderList = []
orderEvent = False
tickSizeToSeconds = {'s1': 1, 's5': 5, 'm1': 60, 'm5': 300}
dataKeys = ['time', 'open', 'high', 'low', 'close', 'volume', 'count']
tickKeys = ['direct', 'takeProfit', 'stopLoss', 'holdPeriod', 'datetime']
FEE = 0.0002 # комиссия
_tickSize = 'm5'

def runBacktest(init, tick, ticker):
    orderList, orderEvent = [], False
    _tickSize, orderList, data = runTick(init, tick, ticker)
    measure = runOrder(ticker, _tickSize, orderList, data)        
    measure['tickFile'] = 'data/order/{0}_{1}_tick.csv'.format(ticker, _tickSize)
    measure['orderFile'] = 'data/order/{0}_{1}_order.csv'.format(ticker, _tickSize)
    print('tickFile =  {0}'.format(measure['tickFile']))
    print('orderFile = {0}'.format(measure['orderFile']))
    return pd.DataFrame([measure], index=[ticker + '_' + _tickSize], 
                                   columns=['sumProcent', 'maxDrawdown', 'std', 
                                            'minV', 'numDeals', 
                                            'sumTakeProfit', 'sumHoldPeriod', 'sumStopLoss'])

def order(direct, takeProfit, stopLoss, holdPeriod):
    global orderList, orderEvent, tickSizeToSeconds, _tickSize
    if not isinstance(holdPeriod, int):
        raise Exception('Hold period must be int type. If you use division with operator /, ' +
                        'remember in python3 this operation converts result to float type, ' +
                        'use // instead or convert to int directly')
    if holdPeriod * tickSizeToSeconds[_tickSize] < 300:
        raise Exception('Hold period must be not less than 300 seconds')
    if takeProfit < 0.0004:
        raise Exception('Take profit must be not less than 0.0004')
    if stopLoss < 0.0004:
        raise Exception('Stop loss must be not less than 0.0004')
    orderList.append([direct, takeProfit, stopLoss, holdPeriod])
    orderEvent = True

def runTick(init, tick, ticker):
    global orderList, orderEvent, _tickSize
    orderList, orderEvent = [], False
    
    class Empty:
        pass
    self = Empty()
    init(self)
    
    _tickSize = getattr(self, '_tickSize', 'm5')
    _window = getattr(self, '_window', None)
    if _window is not None:
        _window = int(_window)
    
    data = {key: np.load('data/{0}/{1}/{2}.npy'.format(ticker, _tickSize, key), encoding='bytes') for key in dataKeys }

    for ind in range(1, len(data['time'])):
        if _window:
            if ind < _window:
                continue
            else:
                tick(self, { key: data[key][ind - _window:ind] for key in dataKeys })
        else:
            tick(self, { key: data[key][:ind] for key in dataKeys })

        if orderEvent:
            for jnd in range(len(orderList) - 1, -1, -1): # [len(orderList) - 1, ..., 0]
                if len(orderList[jnd]) == 4:
                    orderList[jnd].append(data['time'][ind])
                else:
                    break
            orderEvent = False

    with open('data/order/{0}_{1}_tick.csv'.format(ticker, _tickSize), 'w') as file:        
        file.write(';'.join(tickKeys) + '\n')
        for order in orderList:
            file.write(';'.join([str(elem) for elem in order]) + '\n')
            
    return _tickSize, orderList, data

def runOrder(ticker, _tickSize, orderList, dataNpy):
    measure = {'deals': [], 'sumProcent': 0.0, 'sumTakeProfit': 0, 'sumStopLoss': 0, 'sumHoldPeriod': 0, 'numDeals': 0}
    currentDataNum, firstTime, preLastCandle = -1, True, False
    for order in orderList:
        if preLastCandle:
            break
        order = dict(zip(tickKeys, order))        
        
        mode = 'findOrder'
        if firstTime or data['time'] <= order['datetime']:
            while (not preLastCandle) and mode != 'Exit':
                currentDataNum += 1
                if currentDataNum >= len(dataNpy['time']) - 2:
                    preLastCandle = True

                data = {key: dataNpy[key][currentDataNum] for key in dataKeys}

                if mode == 'findOrder':
                    if data['time'] >= order['datetime']:
                        priceEnter = data['close']
                        numEnter = currentDataNum
                        datetimeEnter = data['time']
                        mode = 'doOrder'
                elif mode == 'doOrder':
                    currentDatetime = data['time']
                    procentUp = data['high'] / priceEnter - 1.
                    procentDown = data['low'] / priceEnter - 1.
                    holdPeriod = order['holdPeriod']
                    isHoldPeriod = preLastCandle or (currentDataNum - numEnter + 1 > holdPeriod)                    

                    if order['direct'] == 'buy':
                        takeProfit = (procentUp >= order['takeProfit'])
                        stopLoss = (procentDown <= -order['stopLoss'])
                    else: # order['direct'] == 'sell
                        takeProfit = (procentDown <= -order['takeProfit'])
                        stopLoss = (procentUp >= order['stopLoss'])

                    if takeProfit or stopLoss or isHoldPeriod:
                        event = 'holdPeriod'
                        nextClose = dataNpy['close'][currentDataNum + 1]
                        direct = {'buy': 1, 'sell': -1}[order['direct']]
                        procent = (nextClose / priceEnter - 1.) * direct - 2 * FEE
                        if takeProfit:
                            event = 'takeProfit'                                
                        if stopLoss:
                            event = 'stopLoss'
                        measure['deals'].append({
                            'procent': procent,
                            'event': event,
                            'direct': order['direct'],
                            'datetimeEnter': datetimeEnter,
                            'datetimeExit': currentDatetime,
                            'priceEnter': priceEnter
                        })
                        mode = 'Exit'
        firstTime = False

    mapEventDirect = {'takeProfit': 'sumTakeProfit', 'holdPeriod': 'sumHoldPeriod', 'stopLoss': 'sumStopLoss'}
    portfolio = []    
    for deal in measure['deals']:
        portfolio.append(deal['procent'])
        measure[mapEventDirect[deal['event']]] += deal['procent']

    def calcMeasures(deals):
        def maxDrawdown(array):
            i = np.argmax(np.maximum.accumulate(array) - array) # end of the period
            if i == 0:
                return 0
            j = np.argmax(array[:i]) # start of period
            return array[j] - array[i]
        res = {};
        pnl = np.cumsum(deals)
        res['std'] = np.std(pnl)
        res['minV'] = min(np.min(pnl), 0)
        res['maxDrawdown'] = maxDrawdown(pnl)
        res['sumProcent'] = pnl[-1]
        res['numDeals'] = len(portfolio)
        return res

    measure['sumProcent'] = measure['minV'] = measure['maxDrawdown'] = 0
    measure['std'] = measure['numDeals'] = 0
    if portfolio:
        measureTest = calcMeasures(portfolio)
        measure.update(measureTest)        

    toCSV = [deal for deal in measure['deals']]
    fieldnames = ['datetimeEnter', 'direct', 'priceEnter', 'procent', 'event', 'datetimeExit']
    with open('data/order/{0}_{1}_order.csv'.format(ticker, _tickSize), 'w') as output_file:
        dict_writer = csv.DictWriter(output_file, fieldnames=fieldnames, delimiter=';')
        dict_writer.writeheader()
        dict_writer.writerows(toCSV)
        
    return measure

Для отображения графиков вам потребуется установить зависимости

In [ ]:
import matplotlib.pyplot as plt 
% pylab
ticker = 'SBER'
_tickSize = 'm5'
dataKeys = ['time', 'open', 'high', 'low', 'close', 'volume', 'count']
data = {key: np.load('data/{0}/{1}/{2}.npy'.format(ticker, _tickSize, key), encoding='bytes') for key in dataKeys }
plt.plot(data['time'], data['open'])
plt.show()